In [1]:
invisible(lapply(c(
  "dplyr","tidyr"
), library, character.only = TRUE))

# Set up appropriate contrasts for Type III ANOVAs
## First arg is for unordered factors, second is for ordered
## contr.sum contrast coefficients for each categorical add up to 0
## contr.poly contrasts measures the rate of change of the set of coefficients
options(contrasts = c("contr.sum", "contr.poly"))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## Removing empty ASVs
Some asvs have no counts if you remove the deeper depth and non maps studies

In [2]:
occurances <- read.table(
  file = "occurances.tsv", sep = "\t",
  header = TRUE, check.names = FALSE
)
taxa <- read.table(
  file = "taxa.tsv", sep = "\t",
  header = TRUE, check.names = FALSE
)
totals <- colSums(occurances[!colnames(occurances) %in% c("uid", "sample_id", "marker")])
nonzero_asvs <- names(totals[totals > 0])
new_occurances <- occurances[,colnames(occurances) %in% c("uid", "sample_id", "marker", nonzero_asvs, "total")]
new_taxa <- taxa[taxa$taxa_id %in% nonzero_asvs, ]

write.table(new_occurances, "new_occurances.tsv", sep="\t", row.names=FALSE, quote=FALSE)
write.table(new_taxa, "new_taxa.tsv", sep="\t", row.names=FALSE, quote=FALSE)

In [3]:
occurances <- read.table(
  file = "new_occurances.tsv", sep = "\t",
  header = TRUE, check.names = FALSE
)
taxa <- read.table(
  file = "new_taxa.tsv", sep = "\t",
  header = TRUE, check.names = FALSE
)
occurances <- pivot_longer(
  data = occurances,
  cols = !all_of(c("uid", "sample_id", "marker", "total")),
  names_to = "taxon_id",
  values_to = "abundance"
)
# |> filter(abundance > 0)

# Need to exclude non AMF
nrow(occurances[occurances$taxon_id %in% taxa$taxa_id, "abundance"])
nrow(occurances[, "abundance"])
sum(occurances[occurances$taxon_id %in% taxa$taxa_id, "abundance"])
sum(occurances[, "abundance"])
occurances <- occurances[occurances$taxon_id %in% taxa$taxa_id, ]

write.table(occurances, "m_occurances.tsv", sep = "\t",row.names = FALSE, col.names = TRUE)

[1] 2392

[1] 63664

[1] 2000

[1] 24789